In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_anna_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,"Meet Banu, one of our app developers! She grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1,9,NaN,0.7835,Positive,5
1,1,"#surappa Ji, VC, Anna University CC: @EPSTamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0,0,NaN,0.7650,Positive,5
2,2,"On This Day, Last Year. \n\n""Anna University L...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1,1,NaN,0.0000,Neutral,3
3,3,Anna University will remain a state varsity: O...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0,0,NaN,0.0000,Neutral,3
4,4,Anna University won’t fall under Centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0,0,NaN,0.0000,Neutral,3
5,5,Anna University won’t fall under Centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1,0,NaN,0.0000,Neutral,3
6,6,"On This Day, Last Year. \n\n""Anna University L...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2,1,NaN,0.0000,Neutral,3
7,7,"On This Day, Last Year. \n\n""Anna University L...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2,2,NaN,0.0000,Neutral,3
8,8,Allaying fears that Anna University will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0,0,NaN,-0.4215,Negative,2
9,9,As a resource person for Tamil Nadu Student In...,vigneshwaran.T,1219947886584258560,2020-01-22 11:40:16,"['tamilnadustudentinnovators', 'studentinnovat...",_vigneshwaranT,0,0,NaN,0.0000,Neutral,3


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,"Meet Banu, one of our app developers! She grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1.0,9.0,NaN,0.7835,Positive,5,0.000349,0.000997
1,"#surappa Ji, VC, Anna University CC: @EPSTamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0.0,0.0,NaN,0.7650,Positive,5,0.000000,0.000000
2,"On This Day, Last Year. \n\n""Anna University L...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111
3,Anna University will remain a state varsity: O...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
4,Anna University won’t fall under Centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
5,Anna University won’t fall under Centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1.0,0.0,NaN,0.0000,Neutral,3,0.000349,0.000000
6,"On This Day, Last Year. \n\n""Anna University L...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2.0,1.0,NaN,0.0000,Neutral,3,0.000698,0.000111
7,"On This Day, Last Year. \n\n""Anna University L...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2.0,2.0,NaN,0.0000,Neutral,3,0.000698,0.000222
8,Allaying fears that Anna University will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0.0,0.0,NaN,-0.4215,Negative,2,0.000000,0.000000
9,As a resource person for Tamil Nadu Student In...,vigneshwaran.T,1219947886584258560,2020-01-22 11:40:16,"['tamilnadustudentinnovators', 'studentinnovat...",_vigneshwaranT,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,"meet banu, one of our app developers! she grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1.0,9.0,NaN,0.7835,Positive,5,0.000349,0.000997,0.060807,Meet Banu one of our app developers She gradua...,"[meet, banu, one, app, developers, graduated, ..."
1,"#surappa ji, vc, anna university cc: @epstamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0.0,0.0,NaN,0.7650,Positive,5,0.000000,0.000000,0.000000,#surappa Ji VC Anna University CC @EPSTamilNad...,"[surappa, ji, vc, anna, university, cc, epstam..."
2,"on this day, last year. \n\n""anna university l...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111,0.025365,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th..."
3,anna university will remain a state varsity: o...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University will remain a state varsity Of...,"[anna, university, remain, state, varsity, off..."
4,anna university won’t fall under centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says..."
5,anna university won’t fall under centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1.0,0.0,NaN,0.0000,Neutral,3,0.000349,0.000000,0.020935,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says..."
6,"on this day, last year. \n\n""anna university l...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2.0,1.0,NaN,0.0000,Neutral,3,0.000698,0.000111,0.046300,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th..."
7,"on this day, last year. \n\n""anna university l...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2.0,2.0,NaN,0.0000,Neutral,3,0.000698,0.000222,0.050731,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th..."
8,allaying fears that anna university will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0.0,0.0,NaN,-0.4215,Negative,2,0.000000,0.000000,0.000000,Allaying fears that Anna University will come ...,"[allaying, fears, anna, university, come, cent..."
9,as a resource person for tamil nadu student in...,vigneshwaran.T,1219947886584258560,2020-01-22 11:40:16,"['tamilnadustudentinnovators', 'studentinnovat...",_vigneshwaranT,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,As a resource person for Tamil Nadu Student In...,"[resource, person, tamil, nadu, student, innov..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,"meet banu, one of our app developers! she grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1.0,9.0,NaN,0.7835,Positive,5,0.000349,0.000997,0.060807,Meet Banu one of our app developers She gradua...,"[meet, banu, one, app, developers, graduated, ...","[(meet, NN), (banu, VBD), (one, CD), (app, NN)..."
1,"#surappa ji, vc, anna university cc: @epstamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0.0,0.0,NaN,0.7650,Positive,5,0.000000,0.000000,0.000000,#surappa Ji VC Anna University CC @EPSTamilNad...,"[surappa, ji, vc, anna, university, cc, epstam...","[(surappa, NN), (ji, NN), (vc, NN), (anna, NN)..."
2,"on this day, last year. \n\n""anna university l...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111,0.025365,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)..."
3,anna university will remain a state varsity: o...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University will remain a state varsity Of...,"[anna, university, remain, state, varsity, off...","[(anna, NN), (university, NN), (remain, VBP), ..."
4,anna university won’t fall under centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce..."
5,anna university won’t fall under centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1.0,0.0,NaN,0.0000,Neutral,3,0.000349,0.000000,0.020935,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce..."
6,"on this day, last year. \n\n""anna university l...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2.0,1.0,NaN,0.0000,Neutral,3,0.000698,0.000111,0.046300,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)..."
7,"on this day, last year. \n\n""anna university l...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2.0,2.0,NaN,0.0000,Neutral,3,0.000698,0.000222,0.050731,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)..."
8,allaying fears that anna university will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0.0,0.0,NaN,-0.4215,Negative,2,0.000000,0.000000,0.000000,Allaying fears that Anna University will come ...,"[allaying, fears, anna, university, come, cent...","[(allaying, VBG), (fears, NNS), (anna, IN), (u..."
9,as a resource person for tamil nadu student in...,vigneshwaran.T,1219947886584258560,2020-01-22 11:40:16,"['tamilnadustudentinnovators', 'studentinnovat...",_vigneshwaranT,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,As a resource person for Tamil Nadu Student In...,"[resource, person, tamil, nadu, student, innov...","[(resource, NN), (person, NN), (tamil, JJ), (n..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,"meet banu, one of our app developers! she grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1.0,9.0,NaN,0.7835,Positive,5,0.000349,0.000997,0.060807,Meet Banu one of our app developers She gradua...,"[meet, banu, one, app, developers, graduated, ...","[(meet, NN), (banu, VBD), (one, CD), (app, NN)...","[meet, app, developers, university, bachelor, ..."
1,"#surappa ji, vc, anna university cc: @epstamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0.0,0.0,NaN,0.7650,Positive,5,0.000000,0.000000,0.000000,#surappa Ji VC Anna University CC @EPSTamilNad...,"[surappa, ji, vc, anna, university, cc, epstam...","[(surappa, NN), (ji, NN), (vc, NN), (anna, NN)...","[surappa, ji, vc, anna, university, cc, office..."
2,"on this day, last year. \n\n""anna university l...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111,0.025365,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku..."
3,anna university will remain a state varsity: o...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University will remain a state varsity Of...,"[anna, university, remain, state, varsity, off...","[(anna, NN), (university, NN), (remain, VBP), ...","[anna, university, state, varsity, official, o..."
4,anna university won’t fall under centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce...","[anna, university, fall, centre, control, tn, ..."
5,anna university won’t fall under centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1.0,0.0,NaN,0.0000,Neutral,3,0.000349,0.000000,0.020935,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce...","[anna, university, fall, centre, control, tn, ..."
6,"on this day, last year. \n\n""anna university l...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2.0,1.0,NaN,0.0000,Neutral,3,0.000698,0.000111,0.046300,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku..."
7,"on this day, last year. \n\n""anna university l...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2.0,2.0,NaN,0.0000,Neutral,3,0.000698,0.000222,0.050731,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku..."
8,allaying fears that anna university will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0.0,0.0,NaN,-0.4215,Negative,2,0.000000,0.000000,0.000000,Allaying fears that Anna University will come ...,"[allaying, fears, anna, university, come, cent...","[(allaying, VBG), (fears, NNS), (anna, IN), (u...","[fears, university, centre, secretary, educati..."
9,as a resource person for tamil nadu student in...,vigneshwaran.T,1219947886584258560,2020-01-22 11:40:16,

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,meet
1,app
2,developers
3,university
4,bachelor
5,technology
6,master
7,computer
8,science
9,banu


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,meet,meet,40
1,app,app,20
2,developers,developers,4
3,university,university,4522
4,bachelor,bachelor,7
5,technology,technology,222
6,master,master,43
7,computer,computer,41
8,science,science,113
9,banu,banu,2


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
3,university,university,4522
22,anna,anna,3039
59,annauniversity,annauniversity,2263
17,com,com,1788
16,twitter,twitter,1780
70,college,college,1530
647,computerscience,computerscience,1097
648,kcgtechnology,kcgtechnology,1097
129,chennai,chennai,886
71,students,students,670


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['tamilmemes','bachelorette','funnymemes','funtroll']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
67,who's that vignesh in your friend list? \n.\n....,Insydfeed,1219094138311675904,2020-01-20 03:07:46,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,2.0,NaN,0.4939,Positive,4,0.000349,0.000222,0.029795,Who's that Vignesh in your friend list . . #in...,"[vignesh, friend, list, insydfeed, tamilmemes,...","[(vignesh, JJ), (friend, NN), (list, NN), (ins...","[friend, list, insydfeed, tamilmemes, bachelor...",3
84,mention him ? \n.\n.\n\n#insydfeed #tamilmemes...,Insydfeed,1217722638560153600,2020-01-16 08:17:55,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,3.0,NaN,0.0000,Neutral,3,0.000349,0.000332,0.034226,Mention him . . #insydfeed #tamilmemes #funtro...,"[mention, insydfeed, tamilmemes, funtroll, bac...","[(mention, NN), (insydfeed, NN), (tamilmemes, ...","[mention, insydfeed, tamilmemes, bachelorette,...",2
125,proof \n.\n\n#insydfeed #tamilmemes #funtroll ...,Insydfeed,1255442104261763072,2020-04-29 10:21:37,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,2.0,NaN,0.0000,Neutral,3,0.000349,0.000222,0.029795,Proof . #insydfeed #tamilmemes #funtroll #bach...,"[proof, insydfeed, tamilmemes, funtroll, bache...","[(proof, NN), (insydfeed, NN), (tamilmemes, NN...","[proof, insydfeed, tamilmemes, bachelorette, a...",3
143,true \n.\n.\n#insydfeed #tamilmemes #funtroll ...,Insydfeed,1254816714149126144,2020-04-27 16:56:32,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,2.0,1.0,NaN,0.4215,Positive,4,0.000698,0.000111,0.046300,True . . #insydfeed #tamilmemes #funtroll #bac...,"[true, insydfeed, tamilmemes, funtroll, bachel...","[(true, JJ), (insydfeed, NN), (tamilmemes, NNS...","[insydfeed, tamilmemes, bachelorette, annauniv...",3
144,same pose \n.\n#insydfeed #tamilmemes #vijaytv...,Insydfeed,1254770185237483520,2020-04-27 13:51:39,"['insydfeed', 'tamilmemes', 'vijaytv', 'funtro...",insydfeed,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111,0.025365,Same pose . #insydfeed #tamilmemes #vijaytv #f...,"[pose, insydfeed, tamilmemes, vijaytv, funtrol...","[(pose, JJ), (insydfeed, NN), (tamilmemes, NNS...","[insydfeed, tamilmemes, funtroll, bachelorette...",4
145,guess how mom look at it? \n.\n.\n#insydfeed #...,Insydfeed,1254697049565478912,2020-04-27 09:01:02,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,1.0,NaN,0.0000,Neutral,3,0.000349,0.000111,0.025365,Guess how mom look at it . . #insydfeed #tamil...,"[guess, mom, look, insydfeed, tamilmemes, funt...","[(guess, NN), (mom, NN), (look, VBP), (insydfe...","[guess, mom, tamilmemes, bachelorette, annauni...",3
155,that jealous bestie \n#insydfeed #tamilmemes #...,Insydfeed,1254314129101217792,2020-04-26 07:39:27,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,1.0,NaN,-0.4588,Negative,2,0.000349,0.000111,0.025365,That jealous bestie #insydfeed #tamilmemes #fu...,"[jealous, bestie, insydfeed, tamilmemes, funtr...","[(jealous, JJ), (bestie, NNS), (insydfeed, VBP...","[bestie, tamilmemes, bachelorette, annaunivers...",3
161,90s kid reaction be like \n.\n.\n#insydfeed #t...,Insydfeed,1254082724874186752,2020-04-25 16:19:55,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,2.0,2.0,NaN,0.3612,Positive,4,0.000698,0.000222,0.050731,90s kid reaction be like . . #insydfeed #tamil...,"[90s, kid, reaction, like, insydfeed, tamilmem...","[(90s, CD), (kid, JJ), (reaction, NN), (like, ...","[reaction, insydfeed, tamilmemes, bachelorette...",3
168,think out of the box \n.\n.\n#insydfeed #tamil...,Insydfeed,1253576693639307264,2020-04-24 06:49:08,"['insydfeed', 'tamilmemes', 'funtroll', 'bache...",insydfeed,1.0,3.0,NaN,0.0000,Neutral,3,0.000349,0.000332,0.034226,Think out of the box . . #insydfeed #tamilmeme...,"[think, box, insydfeed, tamilmemes, funtroll, ...","[(think, NN), (box, NN), (insydfeed, VBP),

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']

In [22]:
df_eng1=df[df.engagement==1]

In [23]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df[df.engagement==1]=df_eng1

In [25]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [26]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [27]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,"meet banu, one of our app developers! she grad...",PROSIT Research Lab,1220098927896276992,2020-01-22 21:40:27,[],prosit_lab,1.0,9.0,NaN,0.7835,...,0.000349,0.000997,0.060807,Meet Banu one of our app developers She gradua...,"[meet, banu, one, app, developers, graduated, ...","[(meet, NN), (banu, VBD), (one, CD), (app, NN)...","[meet, app, developers, university, bachelor, ...",0,1,5
1,"#surappa ji, vc, anna university cc: @epstamil...","TNBJP: Rajendran Naidu, ME, FIE.(MP Candidate2...",1220070171441688576,2020-01-22 19:46:11,['surappa'],rajnidihe,0.0,0.0,NaN,0.7650,...,0.000000,0.000000,0.000000,#surappa Ji VC Anna University CC @EPSTamilNad...,"[surappa, ji, vc, anna, university, cc, epstam...","[(surappa, NN), (ji, NN), (vc, NN), (anna, NN)...","[surappa, ji, vc, anna, university, cc, office...",0,0,5
2,"on this day, last year. \n\n""anna university l...",Salmaan_𝟤𝟪𝒴𝓇𝓈𝒪𝒻𝒜𝒿𝒾𝓉𝒽𝒾𝓈𝓂,1220023764458455040,2020-01-22 16:41:47,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",msdsalmaan007,1.0,1.0,NaN,0.0000,...,0.000349,0.000111,0.025365,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku...",0,1,3
3,anna university will remain a state varsity: o...,Orange News,1219993273269284864,2020-01-22 14:40:37,[],MSShanker1,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Anna University will remain a state varsity Of...,"[anna, university, remain, state, varsity, off...","[(anna, NN), (university, NN), (remain, VBP), ...","[anna, university, state, varsity, official, o...",0,0,3
4,anna university won’t fall under centre’s cont...,TOIChennai,1219974277014814720,2020-01-22 13:25:08,[],TOIChennai,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce...","[anna, university, fall, centre, control, tn, ...",0,0,3
5,anna university won’t fall under centre’s cont...,TOI Cities,1219973247724224512,2020-01-22 13:21:02,[],TOICitiesNews,1.0,0.0,NaN,0.0000,...,0.000349,0.000000,0.020935,Anna University won’t fall under Centre’s cont...,"[anna, university, fall, centre, control, says...","[(anna, NN), (university, NN), (fall, NN), (ce...","[anna, university, fall, centre, control, tn, ...",0,1,3
6,"on this day, last year. \n\n""anna university l...",𝚅𝚒𝚐𝚗𝚎𝚜𝚑𝚠𝚊𝚛𝚊𝚗 ツ,1219970605937655808,2020-01-22 13:10:33,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",StuntVicky_Offc,2.0,1.0,NaN,0.0000,...,0.000698,0.000111,0.046300,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku...",0,1,3
7,"on this day, last year. \n\n""anna university l...",🅰️K - T R E N D $ 💯🔥,1219970364773564416,2020-01-22 13:09:35,"['AjithKumar', 'Valimai', 'Thala', 'Ajith', 'E...",ItzmeOffl,2.0,2.0,NaN,0.0000,...,0.000698,0.000222,0.050731,"On This Day Last Year ""Anna University Letter ...","[day, last, year, anna, university, letter, th...","[(day, NN), (last, JJ), (year, NN), (anna, RB)...","[day, year, university, letter, thank, ajithku...",0,1,3
8,allaying fears that anna university will come ...,The Covai Post,1219967554380189696,2020-01-22 12:58:25,"['AnnaUniversity', 'Coimbatore']",CovaiPost,0.0,0.0,NaN,-0.4215,...,0.000000,0.000000,0.000000,Allaying fears that Anna University will come ...,"[allaying, fears, anna, university, come, cent...","[(allaying, VBG), (fears, NNS), (anna, IN), (u...","[fears, university, centre, secretary, educati...",0,0,2
9,as a resource person for tamil nadu student in...,vigneshwaran.T,1219947886584258

In [28]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/AnnaWeightedSentiment.csv')